In [15]:
import os
import pandas as pd 
dataset = pd.read_csv('UCI_Credit_Card.csv', header = 0, encoding='utf-8', engine='python')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [30]:
'''
    Phân chia train/val/train/test
'''
import numpy as np 
model_features = list(set(dataset.columns).difference({"ID", "default.payment.next.month"}))
target = ["default.payment.next.month"]
x = dataset[model_features]
y = dataset[target]

id_pos = np.where(y.values.reshape(-1) == 1)[0]
id_neg = np.where(y.values.reshape(-1) == 0)[0]

np.random.shuffle(id_pos)
np.random.shuffle(id_neg)

#   Tập train
id_train_neg = id_neg[:10000]
id_train_pos = id_pos[:500]
id_train = np.concatenate((id_train_neg, id_train_pos), axis = 0 )

#   Tập val
id_val_neg = id_neg[10000:12000]
id_val_pos = id_pos[500:600]
id_val = np.concatenate((id_val_neg, id_val_pos), axis = 0)


#   Tập dev
id_dev_neg = id_neg[12000:14000]
id_dev_pos = id_pos[600:700]
id_dev = np.concatenate((id_dev_neg, id_dev_pos), axis =0)

#   Tập test
id_test_neg = id_neg[14000:16000]
id_test_pos = id_pos[700:800]
id_test = np.concatenate((id_test_neg, id_test_pos), axis=0)

#   Khởi tạp dataset

data_train = dataset.iloc[id_train]
data_val = dataset.iloc[id_val]
data_dev = dataset.iloc[id_dev]
data_test = dataset.iloc[id_test]

#   Output shape all dataset
print('Shape of data train is: {}'.format(data_train.shape))
print('Shape os data values is: {}'.format(data_val.shape))
print('Shape os data dev is: {}'.format(data_dev.shape))
print('Shape of data test is: {}'.format(data_test.shape))

Shape of data train is: (10500, 25)
Shape os data values is: (2100, 25)
Shape os data dev is: (2100, 25)
Shape of data test is: (2100, 25)


In [17]:
'''
Build Model
'''
import numpy as np 
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import matplotlib.pyplot as plt

In [4]:
model1 = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=200, class_weight=None, max_features=10)

model2 = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_split=400, random_state=12, class_weight='balanced', max_features='auto')

model3 = RandomForestClassifier(n_estimators=800, max_depth=10, min_samples_split=200, random_state=12, class_weight='balanced', max_features='sqrt')

def _tunning_model(model, X_train, y_train, X_dev, y_dev):
    model.fit(X_train, y_train)
    model_predictions = model.predict_proba(X_dev)
    model_roc_score = roc_auc_score(y_dev, model_predictions[:,1])
    return model, model_roc_score

model1, model1_roc_score = _tunning_model(model1, data_train[model_features],
                                                  data_train['default.payment.next.month'],
                                                  data_dev[model_features],
                                                  data_dev['default.payment.next.month'])
print('model 1 ROC score on dev dataset: ', model1_roc_score)

model2, model2_roc_score = _tunning_model(model2, data_train[model_features],
                                                  data_train['default.payment.next.month'],
                                                  data_dev[model_features],
                                                  data_dev['default.payment.next.month'])
print('model 2 ROC score on dev dataset: ', model2_roc_score)

model3, model3_roc_score = _tunning_model(model3, data_train[model_features],
                                                  data_train['default.payment.next.month'],
                                                  data_dev[model_features],
                                                  data_dev['default.payment.next.month'])
print('model 3 ROC score on dev dataset: ', model3_roc_score)

model 1 ROC score on dev dataset:  0.7402074999999999
model 2 ROC score on dev dataset:  0.745485
model 3 ROC score on dev dataset:  0.7486649999999999


In [ ]:
#   Mô hình 2 là mô hình tốt nhất nên chọn mô hình 2 là baseline kèm với nhứng siêu tham số được giữ đểkhởi tạp các mô hình sau

In [34]:
'''
Under Slamping
'''
#   Phan chia mẫu ngẫu nhiên theo tỉ lệ 80:20
id_train_neg_80_20 = id_train_neg[:2000]
id_train_80_20 = np.concatenate((id_train_neg_80_20, id_train_pos), axis=0)

#   Phân chia mẫu ngẫu nhiên theo tỉ lệ 70:30
id_train_neg_70_30 = id_train_neg[:1166]
id_train_70_30 = np.concatenate((id_train_neg_70_30, id_train_pos), axis=0)

#   Khởi tạo dataset 
data_train_80_20 = dataset.iloc[id_train_80_20]
data_train_70_30 = dataset.iloc[id_train_70_30]


In [39]:
#   Huấn luyện mô hình
model2_unsam_80_20 = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_split=400, random_state=12, class_weight="balanced", max_features="auto")
model2_unsam_80_20.fit( data_train_80_20[model_features], data_train_80_20['default.payment.next.month'] )
model_predictions = model2_unsam_80_20.predict_proba(data_test[model_features])
model_pred_label = model2_unsam_80_20.predict(data_test[model_features])
model_roc_score = roc_auc_score(data_test['default.payment.next.month'], model_predictions[:,1])
model_f1_score = f1_score(data_test['default.payment.next.month'], model_pred_label)

print('model2_unsam_80_20 roc score on test: ', model_roc_score)
print('model2_unsam_80_20 f1 score on test: ', model_f1_score)

#   Huấn luyện mô hình 70_30
model2_unsam_70_30 = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_split=400, random_state=12, class_weight="balanced", max_features="auto")
model2_unsam_70_30.fit(data_train_70_30[model_features], data_train_70_30['default.payment.next.month'])
model_predictions = model2_unsam_70_30.predict_proba(data_test[model_features])
model_pred_label = model2_unsam_70_30.predict(data_test[model_features])
model_roc_score = roc_auc_score(data_test['default.payment.next.month'], model_predictions[:,1])
model_f1_score = f1_score(data_test['default.payment.next.month'], model_pred_label)

print('model2_unsam_70_30 roc score on test: ', model_roc_score)
print('model2_unsam_70_30 f1 score on test: ', model_f1_score)


model2_unsam_80_20 roc score on test:  0.8254925000000001
model2_unsam_80_20 f1 score on test:  0.23270440251572327
model2_unsam_70_30 roc score on test:  0.8211350000000001
model2_unsam_70_30 f1 score on test:  0.2191780821917808


In [41]:
'''
Dự báo mô hình trên tập test và tính toán các chỉ số auc và f1 score.
'''
model_predictions = model2.predict_proba(data_test[model_features])
model_pred_label = model2.predict(data_test[model_features]) 
model_roc_score = roc_auc_score(data_test['default.payment.next.month'], model_predictions[:,1])
model_f1_score = f1_score(data_test['default.payment.next.month'], model_pred_label)

print('random forest roc score on test: ', model_roc_score)
print('random forest f1 score on test: ', model_f1_score)

random forest roc score on test:  0.8345350000000001
random forest f1 score on test:  0.2857142857142857


In [45]:
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# # concate nate all layers
# encode_els = concatenate(encode_els) 
inputlayer = Input(shape=(23,))
hidden1 = Dense(units = 128, kernel_initializer = 'normal', activation = 'relu')(inputlayer)
droplayer1 = Dropout(0.8)(hidden1)
hidden2 = Dense(64, kernel_initializer = 'normal', activation = 'relu')(droplayer1)
droplayer2 = Dropout(0.2)(hidden2)
outputlayer = Dense(1, kernel_initializer = 'normal', activation = 'sigmoid')(droplayer2)
mlp_classifier = Model(inputs = inputlayer, outputs = [outputlayer])
mlp_classifier.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               3072      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_______________________________________________________

In [48]:

optimizer = Adam()
mlp_classifier.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
mlp_classifier.fit(data_train[model_features], data_train['default.payment.next.month'],
                   validation_data = (data_val[model_features], data_val['default.payment.next.month']),
                   epochs = 10,
                   batch_size = 64)

Epoch 1/10
165/165 [==============================] - 1s 2ms/step - loss: 192.1925 - accuracy: 0.8990 - val_loss: 21.6276 - val_accuracy: 0.9524
Epoch 2/10
165/165 [==============================] - 0s 1ms/step - loss: 33.5148 - accuracy: 0.9064 - val_loss: 8.0490 - val_accuracy: 0.9524
Epoch 3/10
165/165 [==============================] - 0s 1ms/step - loss: 14.8427 - accuracy: 0.9163 - val_loss: 3.7490 - val_accuracy: 0.9524
Epoch 4/10
165/165 [==============================] - 0s 1ms/step - loss: 6.8975 - accuracy: 0.9244 - val_loss: 1.5071 - val_accuracy: 0.9519
Epoch 5/10
165/165 [==============================] - 0s 1ms/step - loss: 3.8640 - accuracy: 0.9299 - val_loss: 1.0088 - val_accuracy: 0.9524
Epoch 6/10
165/165 [==============================] - 0s 2ms/step - loss: 2.5673 - accuracy: 0.9308 - val_loss: 0.6978 - val_accuracy: 0.9490
Epoch 7/10
165/165 [==============================] - 0s 1ms/step - loss: 1.5434 - accuracy: 0.9348 - val_loss: 0.5453 - val_accuracy: 0.9524
E